In [3]:
import numpy as np
import os
import pickle
from pathlib import Path
import requests
import string
import re
from datasets import load_dataset

def decode(l):
    return ''.join([itos[i] for i in l])


# NLTK function to generate ngrams
import nltk
from nltk.util import ngrams

Questions:

- remove the speaker for Shakespeare (currently, no)
- remove the title for Wiki text (currently, yes) 
- are the documents just appended to each other? 

# HELPER FUNCTIONS

In [74]:
def average_branching_factor(text, n):
    words = nltk.word_tokenize(text)
    n_grams = list(ngrams(words, n))
    unique_n_grams = list(set(n_grams))
    words = list(set(words))
    total_branches = np.zeros(len(words))

    for i in range(0, len(words)): 
        word = words[i]
        for gram in unique_n_grams:
            if gram[0] == word:
                total_branches[i] += 1
        
    branch_dict = dict(keys = words, values = total_branches)


    return np.mean(total_branches)

In [188]:
def average_branching_factor_split(text, n):
    words = nltk.word_tokenize(text)
    n_grams = list(ngrams(words, n))
    unique_n_grams = list(set(n_grams))
    words = list(set(words))
    total_branches = np.zeros(len(words))

    for i in range(0, len(words)): 
        word = words[i]
        if word != "ENDEMAILHERENJJ": 
            for gram in unique_n_grams:
                if gram[0] == word and gram[1] != "ENDEMAILHERENJJ":
                    total_branches[i] += 1
        
        branch_dict = dict(keys = words, values = total_branches)
    return sum(total_branches)/(len(words)-1) #don't count stop word as a word

In [205]:
#entropy 

def get_entropy(text):
    words = nltk.word_tokenize(text)
    unique_words = list(set(words))
    freq_array = np.zeros(len(unique_words))
    for i in range(0, len(unique_words)):
        word = unique_words[i]
        freq_array[i] = words.count(word)
    p = freq_array/len(unique_words)
    entropy = np.sum(np.multiply(p*-1, np.log2(p))) 
    return entropy

In [1]:
def get_entropy_split(text):
    words = nltk.word_tokenize(text) #remove split 
    words = [x for x in words if x != "ENDEMAILHERENJJ"] #remove splitting string for this 
    print(words)
    unique_words = list(set(words))
    freq_array = np.zeros(len(unique_words))
    for i in range(0, len(unique_words)):
        word = unique_words[i]
        freq_array[i] = words.count(word)
    p = freq_array/len(unique_words)
    entropy = np.sum(np.multiply(p*-1, np.log2(p))) 
    return entropy

# SHAKESPEARE

In [217]:
with open("data/shakespeare/input.txt", 'r', encoding='utf-8') as f:
    data = f.read()

In [218]:
shakespeare = data.translate(str.maketrans("","", string.punctuation)).replace("\n", " ").lower()

In [9]:
# Pointwise Information Matrix 

from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(word_tokenize(data2))

[nltk_data] Downloading package punkt to /Users/jaylincl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
len(finder.score_ngrams(bigram_measures.pmi)) #check vocab size 
#this only bigram PMI, doesn't give us one for any words that don't occur next to each other 

106835

In [219]:
#VOCAB SIZE 
shakespeare_words = nltk.word_tokenize(shakespeare)
len(set(shakespeare_words))

12847

In [76]:
average_branching_factor(data2, 2)

8.315949248851872

In [220]:
shakespeare_entropy = get_entropy(data2)
shakespeare_entropy

90.76569383147365

# WikiText 

In [4]:
wiki = load_dataset("iohadrubin/wikitext-103-raw-v1", num_proc=8)

Found cached dataset parquet (/Users/jaylincl/.cache/huggingface/datasets/iohadrubin___parquet/iohadrubin--wikitext-103-raw-v1-a0b8bc67accf87c3/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [86]:
len(wiki["train"]["text"]) #29567 documents 

wiki_train = wiki["train"]["text"]

In [5]:
def process_wiki(text_list):
    processed = ''

    for i in range(0, len(text_list)):
        doc = text_list[i]
        #remove title 
        without_title = re.sub(r'=[^\n]+\n', '', doc, count = 1)
        #remove foreign characters, numbers, punctuation, line breaks, and go to lowercase
        processed += re.sub(r'[^a-zA-z ]', '', without_title).translate(
            str.maketrans("","", string.punctuation)).replace(
            "\n", " ").lower()
        processed += " ENDEMAILHERENJJ"
    return processed

In [6]:
wiki_train_processed = process_wiki(wiki['train']['text'])
wiki_val_processed = process_wiki(wiki['validation']['text'])
wiki_test_processed = process_wiki(wiki['test']['text'])

In [7]:
wiki_processed = wiki_train_processed + wiki_val_processed + wiki_test_processed

In [8]:
wiki_words = nltk.word_tokenize(wiki_processed)
len(set(wiki_words))-1

970404

In [ ]:
#RUN OVERNIGHT 
wiki_abf = average_branching_factor_split(wiki_processed, 2)
wiki_abf

In [ ]:
wiki_entropy = get_entropy_split(wiki_processed)
wiki_entropy

## ENRON 

In [150]:
enron = load_dataset("snoop2head/enron_aeslc_emails", num_proc=8)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/535703 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /Users/jaylincl/.cache/huggingface/datasets/snoop2head___parquet/snoop2head--enron_aeslc_emails-fc92fd888f9d8151/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [171]:
def process_enron(text_list):
    processed = ''

    for i in range(0, len(text_list)):
        doc = text_list[i]
        #remove title 
        body = doc.partition("Body:")[2]
        #remove foreign characters, numbers, punctuation, line breaks, and go to lowercase
        processed += re.sub(r'[^a-zA-z ]', '', body).translate(
            str.maketrans("","", string.punctuation)).replace(
            "\n", " ").lower()
        processed += " ENDEMAILHERENJJ"
    return processed

In [173]:
enron_processed = process_enron(enron["train"]["text"])

In [174]:
enron_words = nltk.word_tokenize(enron_processed)
len(set(enron_words))-1

1619263

In [ ]:
#RUN OVERNIGHT
enron_abf = average_branching_factor_split(enron_processed, 2)
enron_abf

In [ ]:
enron_entropy = get_entropy_split(enron_processed)
enron_entropy